# Caching VLLM

- 著者: [Joseph](https://github.com/XaviereKU)
- ピアレビュー : [Teddy Lee](https://github.com/teddylee777), [BAEM1N](https://github.com/BAEM1N)
- 校閲 : [Two-Jay](https://github.com/Two-Jay)
- これは [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial) の一部です

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/04-Model/03-Cache_vllm.ipynb)

## 概要

```LangChain``` は LLM 用のオプショナルなキャッシュ層を提供します。

これは次の二つの理由で有用です:
- 同じ補完（completion）を複数回要求する場合、LLM プロバイダへの API 呼び出し回数を減らし、コストを節約できます。
- LLM プロバイダへの API 呼び出し回数を減らすことで、アプリケーションの実行時間（wall time）を改善できます。

しかし、クラウドサービスにアクセスできないオンプレミス環境など、自分で LLM サービスをデプロイする必要がある場合もあります。
このチュートリアルでは OpenAI 互換の API を提供する ```vllm``` を使い、```InMemoryCache``` と ```SQLiteCache``` の2種類のキャッシュを利用します。
各セクションの最後でキャッシュ適用前後の実行時間（wall time）を比較します。

このチュートリアルはローカル LLM サービスの例が中心ですが、まず OpenAI API サービスでのキャッシュの使い方を確認します。

### 目次

- [概要](#overview)
- [環境セットアップ](#environment-setup)
- [InMemoryCache](#inmemorycache)
- [SQlite Cache](#sqlite-cache)
- [VLLM を使ったローカル LLM のセットアップ](#setup-local-llm-with-vllm)
- [InMemoryCache + ローカル VLLM](#inmemorycache--local-vllm)
- [SQLite Cache + ローカル VLLM](#sqlite-cache--local-vllm)

### 参考
- [SQLIteCache](https://python.langchain.com/api_reference/community/cache/langchain_community.cache.SQLiteCache.html#langchain_community.cache.SQLiteCache)
- [InMemoryCache](https://python.langchain.com/api_reference/core/caches/langchain_core.caches.InMemoryCache.html)
- [vLLM](https://docs.vllm.ai/en/latest/)
----


## Environment Setup

環境をセットアップします。詳細については [Environment Setup](https://wikidocs.net/257836) を参照してください。

**[注意]**
- ```langchain-opentutorial``` はチュートリアル用に便利な環境セットアップやユーティリティを提供するパッケージです。
- 詳細は [```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) を参照してください。


In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_core",
        "langchain_community",
        "langchain_openai",
        # "vllm", # this is for optional section
    ],
    verbose=False,
    upgrade=False,
)

In [3]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "You OpenAI API KEY",
        "LANGCHAIN_API_KEY": "LangChain API KEY",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "Caching",
    }
)

Environment variables have been set successfully.


In [4]:
# Alternatively, one can set environmental variables with load_dotenv
from dotenv import load_dotenv


load_dotenv(override=True)

False

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

# Create model
llm = ChatOpenAI(model_name="gpt-4o-mini")

# Generate prompt
prompt = PromptTemplate.from_template(
    "Sumarize about the {country} in about 200 characters"
)

# Create chain
chain = prompt | llm

In [6]:
%%time
# Invoke chain
response = chain.invoke({"country": "South Korea"})
print(response.content)

South Korea, located on the Korean Peninsula, is known for its rich culture, technological advancements, and vibrant economy. It features a mix of traditional heritage and modern innovation, hi[...]
CPU times: total: 93.8 ms
Wall time: 1.09 s


## ```InMemoryCache```
まず、同じ質問に対する応答を ```InMemoryCache``` にキャッシュします。


In [18]:
from langchain_core.globals import set_llm_cache
from langchain_core.caches import InMemoryCache

# Set InMemoryCache
set_llm_cache(InMemoryCache())

In [8]:
%%time
# Invoke chain
response = chain.invoke({"country": "South Korea"})
print(response.content)

South Korea is a technologically advanced country known for its fast-paced lifestyle, vibrant culture, and delicious cuisine. It is a leader in industries such as electronics, automotive, and e[...]
CPU times: total: 0 ns
Wall time: 996 ms


同じ質問でチェーンを再度呼び出します。

In [10]:
%%time
# Invoke chain
response = chain.invoke({"country": "South Korea"})
print(response.content)

South Korea is a technologically advanced country known for its fast-paced lifestyle, vibrant culture, and delicious cuisine. It is a leader in industries such as electronics, automotive, and e[...]
CPU times: total: 0 ns
Wall time: 3 ms


注意: もしもう一度 ```InMemoryCache``` を設定すると、キャッシュは失われ、wall time が増加します。

In [11]:
set_llm_cache(InMemoryCache())

In [12]:
%%time
# Invoke chain
response = chain.invoke({"country": "South Korea"})
print(response.content)

South Korea is a tech-savvy, modern country known for its vibrant culture, delicious cuisine, and booming economy. It is a highly developed nation with advanced infrastructure, high standards o[...]
CPU times: total: 0 ns
Wall time: 972 ms


## ```SQLiteCache```
次に、同じ質問に対する応答を ```SQLiteCache``` を使ってキャッシュします。

In [13]:
from langchain_community.cache import SQLiteCache
from langchain_core.globals import set_llm_cache
import os

# Create cache directory
if not os.path.exists("cache"):
    os.makedirs("cache")

# Set SQLiteCache
set_llm_cache(SQLiteCache(database_path="cache/llm_cache.db"))

In [14]:
%%time
# Invoke chain
response = chain.invoke({"country": "South Korea"})
print(response.content)

South Korea is a technologically advanced country in East Asia, known for its booming economy, vibrant pop culture, and rich history. It is home to K-pop, Samsung, and delicious cuisine like ki[...]
CPU times: total: 31.2 ms
Wall time: 953 ms


同じ質問でチェーンを再度呼び出します。

In [15]:
%%time
# Invoke chain
response = chain.invoke({"country": "South Korea"})
print(response.content)

South Korea is a technologically advanced country in East Asia, known for its booming economy, vibrant pop culture, and rich history. It is home to K-pop, Samsung, and delicious cuisine like ki[...]
CPU times: total: 375 ms
Wall time: 375 ms


注意: ```SQLiteCache``` を使用した場合、キャッシュを再設定しても保存されたキャッシュは削除されません。

In [16]:
set_llm_cache(SQLiteCache(database_path="cache/llm_cache.db"))

In [17]:
%%time
# Invoke chain
response = chain.invoke({"country": "South Korea"})
print(response.content)

South Korea is a technologically advanced country in East Asia, known for its booming economy, vibrant pop culture, and rich history. It is home to K-pop, Samsung, and delicious cuisine like ki[...]
CPU times: total: 0 ns
Wall time: 4.01 ms


## VLLM を使ったローカル LLM のセットアップ
vLLM は様々なケースをサポートしますが、最も安定したセットアップのために ```docker``` を使ってローカルで LLM モデルを ```vLLM``` で提供する手順を利用します。

### デバイス & サービング情報 - Windows
- CPU : AMD 5600X
- OS : Windows 10 Pro
- RAM : 32 Gb
- GPU : Nividia 3080Ti, 12GB VRAM
- CUDA : 12.6
- ドライババージョン : 560.94
- Docker イメージ : nvidia/cuda:12.4.1-cudnn-devel-ubuntu20.04
- モデル : Qwen/Qwen2.5-0.5B-Instruct
- Python バージョン : 3.10
- docker run スクリプト :
    ```
    docker run -itd --name vllm --gpus all --entrypoint /bin/bash -p 6001:8888 nvidia/cuda:12.4.1-cudnn-devel-ubuntu20.04
    ```
- vllm サービング スクリプト : 
    ```
    python3 -m vllm.entrypoints.openai.api_server --model='Qwen/Qwen2.5-0.5B-Instruct' --served-model-name 'qwen-2.5' --port 8888 --host 0.0.0.0 --gpu-memory-utilization 0.80 --max-model-len 4096[...]
    ```


In [18]:
from langchain_community.llms import VLLMOpenAI

# create model using OpenAI compatible class VLLMOpenAI
llm = VLLMOpenAI(
    model="qwen-2.5", openai_api_key="EMPTY", openai_api_base="http://localhost:6001/v1"
)

# Generate prompt
prompt = PromptTemplate.from_template(
    "Sumarize about the {country} in about 200 characters"
)

# Create chain
chain = prompt | llm

## InMemoryCache + ローカル VLLM
上の ```InMemoryCache``` セクションと同様に、ここでも ```InMemoryCache``` を設定します。


In [19]:
from langchain_core.globals import set_llm_cache
from langchain_core.caches import InMemoryCache

# Set InMemoryCache
set_llm_cache(InMemoryCache())

ローカル LLM を使ってチェーンを呼び出します。`response.content` ではなく `response` 全体を print している点に注意してください。

In [20]:
%%time
# Invoke chain
response = chain.invoke({"country": "South Korea"})
print(response)

.
South Korea is a country in East Asia, with a population of approximately 55.2 million as of 2023. It borders North Korea to the east, Japan to the northeast, and China to the southeast. The co[...]
CPU times: total: 15.6 ms
Wall time: 1.03 s


同じ質問でチェーンをもう一度呼び出します。

In [21]:
%%time
# Invoke chain
response = chain.invoke({"country": "South Korea"})
print(response)

.
South Korea is a country in East Asia, with a population of approximately 55.2 million as of 2023. It borders North Korea to the east, Japan to the northeast, and China to the southeast. The co[...]
CPU times: total: 0 ns
Wall time: 2.61 ms


## SQLite Cache + ローカル VLLM
上の ```SQLiteCache``` セクションと同様に、ここでも ```SQLiteCache``` を設定します。
ここでは、```SQLiteCache``` セクションで使ったキャッシュと区別するためにデータベース名を ```vllm_cache.db``` に設定している点に注意してください。


In [23]:
from langchain_community.cache import SQLiteCache
from langchain_core.globals import set_llm_cache
import os

# Create cache directory
if not os.path.exists("cache"):
    os.makedirs("cache")

# Set SQLiteCache
set_llm_cache(SQLiteCache(database_path="cache/vllm_cache.db"))

ローカル LLM を使ってチェーンを呼び出します。ここでも `response.content` ではなく `response` 全体を print している点に注意してください。

In [24]:
%%time
# Invoke chain
response = chain.invoke({"country": "South Korea"})
print(response)

.

South Korea, a nation that prides itself on its history, culture, and natural beauty. Known for its bustling cityscapes, scenic valleys, and delicious cuisine. A major player in South East Asia[...]
CPU times: total: 0 ns
Wall time: 920 ms


同じ質問でチェーンを再度呼び出します。

In [25]:
%%time
# Invoke chain
response = chain.invoke({"country": "South Korea"})
print(response)

.

South Korea, a nation that prides itself on its history, culture, and natural beauty. Known for its bustling cityscapes, scenic valleys, and delicious cuisine. A major player in South East Asia[...]
CPU times: total: 0 ns
Wall time: 3 ms
